In [1]:
import pandas as pd
import re
import io
from unicodedata import normalize
from sqlalchemy import create_engine
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Core
import numpy as np
import scipy
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab


pd.set_option('display.max_columns', None)

data = pd.read_csv("../data/jorge_renovacion.csv", low_memory=False, encoding='latin-1')

In [29]:
# principal features 
print(data.shape)
data.head()

(816687, 50)


,alumno,fecha_inicio_primer_contrato,fecha_fin_primer_contrato,edad,countryname,renueva,timezone,genero,tutor_principal,id_sesion,fecha,dia_contrato,estado_sesion,tipo_sesion,efectividad,tmr,unidades_superadas,unidades_superadas_sesiones,unidades_superadas_test_avance,unidades_superadas_test_entrada,estado_correccion,num_problemas_corregidos,num_problemas_correctos_corregidos,efectividad_correccion,mundo_virtual,calificacion,num_problemas,pregunta_inicio,pregunta_fin,tipo_app,racha_20,racha_30,racha_50,contenido_avanzado,completado_plan_estudios,num_partidas,num_juegos,num_competiciones,tiempo_dedicado_conteo_y_numeros_cardinales,tiempo_dedicado_expresiones_y_ecuaciones,tiempo_dedicado_geometria,tiempo_dedicado_medicion_y_datos,tiempo_dedicado_numeros_enteros,tiempo_dedicado_numeros_y_operaciones__decimales,tiempo_dedicado_numeros_y_operaciones__en_base_diez,tiempo_dedicado_numeros_y_operaciones__fracciones,tiempo_dedicado_operaciones_y_pensamiento_algebraico,dia_semana,hora_inicio,hora_fin
0,1111722,2019-11-10,2020-02-09,9,United Kingdom,1,Europe/London,MASCULINO,653368,43167585,2020-01-15,66,NO_REALIZADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO_REALIZADA,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,desktop,0,0,0,0,65.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wednesday,NaN,NaN
1,2005094,2020-04-13,2020-07-12,10,Spain,0,Europe/Madrid,MASCULINO,1086291,52009955,2020-05-09,26,NO_REALIZADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO_REALIZADA,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,0,0,0,0,64.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,NaN,NaN
2,2514586,2020-05-31,2020-08-30,7,Spain,1,Europe/Madrid,FEMENINO,1397456,58041026,2020-08-02,63,NO_REALIZADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO_REALIZADA,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,0,0,0,0,50.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,NaN,NaN
3,2429101,2020-05-16,2020-08-15,3,Spain,1,Europe/Madrid,FEMENINO,618476,58859740,2020-08-15,91,NO_REALIZADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO_REALIZADA,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,NaN,0,0,0,0,9.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,NaN,NaN
4,1649868,2019-12-12,2020-03-11,8,Spain,1,Europe/Madrid,FEMENINO,237948,43981849,2020-02-08,58,NO_REALIZADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO_REALIZADA,NaN,NaN,NaN,0,NaN,0.0,NaN,NaN,desktop,0,0,0,0,44.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Saturday,NaN,NaN


In [2]:
# set date format 
data["fecha_inicio_primer_contrato"] = pd.to_datetime(data["fecha_inicio_primer_contrato"])
data["fecha_fin_primer_contrato"] = pd.to_datetime(data["fecha_fin_primer_contrato"])
data["fecha"] = pd.to_datetime(data["fecha"])

# modify timezone 
data["zone"] = [re.sub(r'(/)\w+', r'', str(x)) for x in data.timezone]
data["zone"].value_counts()

Europe       711449
America       81117
Atlantic       9510
Africa         8046
Asia           3934
Australia      1458
Pacific         455
nan             443
Indian          275
Name: zone, dtype: int64

In [3]:
# null inspection 
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 816687 entries, 0 to 816686
Data columns (total 51 columns):
 #   Column                                                Non-Null Count   Dtype         
---  ------                                                --------------   -----         
 0   alumno                                                816687 non-null  int64         
 1   fecha_inicio_primer_contrato                          816687 non-null  datetime64[ns]
 2   fecha_fin_primer_contrato                             816687 non-null  datetime64[ns]
 3   edad                                                  816687 non-null  int64         
 4   countryname                                           816244 non-null  object        
 5   renueva                                               816687 non-null  int64         
 6   timezone                                              816244 non-null  object        
 7   genero                                                816687 non-

In [32]:
# state counts
data.estado_sesion.value_counts()

REALIZADA       559759
NO_REALIZADA    235655
INICIADA         19502
VACACIONES        1771
Name: estado_sesion, dtype: int64

In [5]:
# we choose only the sessions performed and initiated. The others have no relevant information. 
data2 = data[data.estado_sesion.isin(["REALIZADA", "INICIADA"]) ]

data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 579261 entries, 235681 to 816686
Data columns (total 51 columns):
 #   Column                                                Non-Null Count   Dtype         
---  ------                                                --------------   -----         
 0   alumno                                                579261 non-null  int64         
 1   fecha_inicio_primer_contrato                          579261 non-null  datetime64[ns]
 2   fecha_fin_primer_contrato                             579261 non-null  datetime64[ns]
 3   edad                                                  579261 non-null  int64         
 4   countryname                                           579018 non-null  object        
 5   renueva                                               579261 non-null  int64         
 6   timezone                                              579018 non-null  object        
 7   genero                                                579261

In [47]:
data2.head()

,alumno,fecha_inicio_primer_contrato,fecha_fin_primer_contrato,edad,countryname,renueva,timezone,genero,tutor_principal,id_sesion,fecha,dia_contrato,estado_sesion,tipo_sesion,efectividad,tmr,unidades_superadas,unidades_superadas_sesiones,unidades_superadas_test_avance,unidades_superadas_test_entrada,estado_correccion,num_problemas_corregidos,num_problemas_correctos_corregidos,efectividad_correccion,mundo_virtual,calificacion,num_problemas,pregunta_inicio,pregunta_fin,tipo_app,racha_20,racha_30,racha_50,contenido_avanzado,completado_plan_estudios,num_partidas,num_juegos,num_competiciones,tiempo_dedicado_conteo_y_numeros_cardinales,tiempo_dedicado_expresiones_y_ecuaciones,tiempo_dedicado_geometria,tiempo_dedicado_medicion_y_datos,tiempo_dedicado_numeros_enteros,tiempo_dedicado_numeros_y_operaciones__decimales,tiempo_dedicado_numeros_y_operaciones__en_base_diez,tiempo_dedicado_numeros_y_operaciones__fracciones,tiempo_dedicado_operaciones_y_pensamiento_algebraico,dia_semana,hora_inicio,hora_fin
235681,1401285,2019-09-26,2019-12-25,7,Spain,1,Europe/Madrid,MASCULINO,792149,41397728,2019-11-14,49,REALIZADA,CODING,NaN,NaN,0.0,0.0,0.0,0.0,NO_REALIZADA,NaN,NaN,NaN,1,NaN,0.0,0.0,5.0,ios,0,0,0,0,43.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,2019-11-14 19:16:42,2019-11-14 19:37:08
235682,1755704,2020-02-12,2020-05-11,7,Spain,1,Europe/Madrid,FEMENINO,956079,45079283,2020-03-08,25,REALIZADA,CODING,0.86,NaN,0.0,0.0,0.0,0.0,NO_REALIZADA,NaN,NaN,NaN,1,NaN,14.0,4.0,5.0,android,0,0,0,0,47.0,20,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,2020-03-08 19:13:34,2020-03-08 19:36:27
235683,1450884,2019-10-07,2020-01-06,7,Spain,1,Europe/Madrid,FEMENINO,815561,41855665,2019-11-28,52,REALIZADA,CODING,0.47,NaN,0.0,0.0,0.0,0.0,NO_REALIZADA,NaN,NaN,NaN,1,NaN,15.0,5.0,3.0,android,0,0,0,0,45.0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Thursday,2019-11-28 19:35:44,2019-11-28 19:54:19
235684,1704515,2020-01-26,2020-04-25,7,Spain,1,Europe/Madrid,FEMENINO,931741,44249962,2020-02-16,21,REALIZADA,CODING,0.58,NaN,0.0,0.0,0.0,0.0,NO_REALIZADA,NaN,NaN,NaN,1,NaN,19.0,5.0,5.0,android,0,0,0,0,52.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sunday,2020-02-16 11:16:23,2020-02-16 11:34:20
235685,1617643,2019-12-03,2020-03-02,10,Spain,1,Europe/Madrid,MASCULINO,335564,42889523,2020-01-06,34,REALIZADA,CODING,0.86,NaN,0.0,0.0,0.0,0.0,NO_REALIZADA,NaN,NaN,NaN,1,NaN,22.0,3.0,3.0,android,0,0,0,0,66.0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Monday,2020-01-06 10:41:39,2020-01-06 11:00:18


In [6]:
# sesion type counts
data2.tipo_sesion.value_counts()

NORMAL          226465
INFANTIL        152386
TUTORIAL         47748
LOGICA           47231
EXAMEN           37785
CODING           33056
TEST_AVANCE      20379
TEST_ENTRADA     14211
Name: tipo_sesion, dtype: int64

In [36]:
##  null inspection 

# null efectitivdad. 
print("null distribution of 'efectividad': \n", data2[data2.efectividad.isnull()]["tipo_sesion"].value_counts())

# null tmr 
print("\n \n null distribution of 'tmr' by 'tipo_sesion': \n", data2[data2.tmr.isnull()]["tipo_sesion"].value_counts())
print("\n \n null distribution of 'tmr' by 'estado_sesion': \n", data2[data2.tmr.isnull()]["estado_sesion"].value_counts())

# null num_problemas_corregidos
print("\n \n null distribution of 'num_problemas_corregidos' by 'tipo_sesion': \n", data2[data2.num_problemas_corregidos.isnull()]["tipo_sesion"].value_counts())
print("\n \n null distribution of 'num_problemas_corregidos' by 'estado_sesion': \n", data2[data2.num_problemas_corregidos.isnull()]["estado_sesion"].value_counts())

# null calificacion
print("\n \n null distribution of 'calificacion' by 'tipo_sesion': \n", data2[data2.calificacion.isnull()]["tipo_sesion"].value_counts())
print("\n \n null distribution of 'calificacion' by 'estado_sesion': \n", data2[data2.calificacion.isnull()]["estado_sesion"].value_counts())
# sesiones d coding y de logica no tienen calificacion? 

## preguntar por que puede ser!!! 
# pregunta inicio y pregunta fin son opcionales. 


null distribution of 'efectividad': 
 CODING    1670
Name: tipo_sesion, dtype: int64

 
 null distribution of 'tmr' by 'tipo_sesion': 
 CODING          33056
NORMAL           1555
INFANTIL          990
TUTORIAL          786
LOGICA            180
TEST_ENTRADA      170
TEST_AVANCE       148
EXAMEN            102
Name: tipo_sesion, dtype: int64

 
 null distribution of 'tmr' by 'estado_sesion': 
 REALIZADA    32043
INICIADA      4944
Name: estado_sesion, dtype: int64

 
 null distribution of 'num_problemas_corregidos' by 'tipo_sesion': 
 NORMAL          72399
INFANTIL        40451
CODING          33056
TUTORIAL        14005
EXAMEN          12387
LOGICA          12094
TEST_AVANCE      4979
TEST_ENTRADA     4273
Name: tipo_sesion, dtype: int64

 
 null distribution of 'num_problemas_corregidos' by 'estado_sesion': 
 REALIZADA    174142
INICIADA      19502
Name: estado_sesion, dtype: int64

 
 null distribution of 'calificacion' by 'tipo_sesion': 
 LOGICA          47231
CODING          33056

In [12]:
# summary of features by student

prueba = data2.groupby('alumno').agg({'fecha_inicio_primer_contrato':'max',
                                    'fecha_fin_primer_contrato':'max', 
                                    'edad':'max',
                                    'genero':'max',
                                    'zone':'max',
                                    'renueva':'max',
                                    'tutor_principal':'max',
                                    'id_sesion':'count', 
                                    'efectividad':'mean', 
                                    'tmr':'mean',
                                    'unidades_superadas':'sum',
                                    'unidades_superadas_sesiones':'sum',
                                    'unidades_superadas_test_avance':'sum', 
                                    'unidades_superadas_test_entrada':'sum',
                                    'num_problemas':'mean', 
                                    'num_problemas_corregidos':'sum',
                                    'num_problemas_correctos_corregidos':'sum',
                                    'efectividad_correccion':'mean',
                                    'mundo_virtual':'mean',                                          
                                    'pregunta_inicio':'mean',
                                    'pregunta_fin':'mean', 
                                    'racha_20':'mean',
                                    'racha_30':'mean',
                                    'racha_50':'mean',
                                    'completado_plan_estudios':'max',
                                    'num_partidas':'mean', 
                                    'num_juegos':'mean', 
                                    'num_competiciones':'mean', 
                                    'tiempo_dedicado_conteo_y_numeros_cardinales':'sum',
                                    'tiempo_dedicado_expresiones_y_ecuaciones':'sum',
                                    'tiempo_dedicado_geometria':'sum',
                                    'tiempo_dedicado_medicion_y_datos':'sum',
                                    'tiempo_dedicado_numeros_enteros':'sum',
                                    'tiempo_dedicado_numeros_y_operaciones__decimales':'sum',
                                    'tiempo_dedicado_numeros_y_operaciones__en_base_diez':'sum',
                                    'tiempo_dedicado_numeros_y_operaciones__fracciones':'sum',
                                    'tiempo_dedicado_operaciones_y_pensamiento_algebraico':'sum'
                                     }).reset_index() 

In [13]:
prueba.head()

,alumno,fecha_inicio_primer_contrato,fecha_fin_primer_contrato,edad,genero,zone,renueva,tutor_principal,id_sesion,efectividad,tmr,unidades_superadas,unidades_superadas_sesiones,unidades_superadas_test_avance,unidades_superadas_test_entrada,num_problemas,num_problemas_corregidos,num_problemas_correctos_corregidos,efectividad_correccion,mundo_virtual,pregunta_inicio,pregunta_fin,racha_20,racha_30,racha_50,completado_plan_estudios,num_partidas,num_juegos,num_competiciones,tiempo_dedicado_conteo_y_numeros_cardinales,tiempo_dedicado_expresiones_y_ecuaciones,tiempo_dedicado_geometria,tiempo_dedicado_medicion_y_datos,tiempo_dedicado_numeros_enteros,tiempo_dedicado_numeros_y_operaciones__decimales,tiempo_dedicado_numeros_y_operaciones__en_base_diez,tiempo_dedicado_numeros_y_operaciones__fracciones,tiempo_dedicado_operaciones_y_pensamiento_algebraico
0,36220,2020-01-14,2020-04-13,4,FEMENINO,Europe,1,903671,55,0.735455,1.180417,68.0,13.0,8.0,47.0,57.472727,170.0,134.0,0.785652,0.781818,2.680000,3.840909,0.563636,0.381818,0.054545,54.0,8.800000,1.381818,0.181818,0.0,0.0,957.0,467.0,0.0,0.0,2800.0,5518.0,24324.0
1,57552,2020-03-03,2020-06-02,3,FEMENINO,Europe,1,45711,68,0.715441,1.146393,84.0,34.0,0.0,50.0,46.573529,106.0,75.0,0.660000,0.911765,4.206349,4.844828,0.250000,0.073529,0.029412,45.0,0.500000,0.147059,0.132353,0.0,0.0,1355.0,2563.0,0.0,0.0,5701.0,777.0,33072.0
2,64537,2020-01-15,2020-04-14,6,MASCULINO,Europe,1,49627,81,0.746173,1.029054,65.0,25.0,5.0,35.0,68.530864,255.0,172.0,0.747368,0.802469,2.716049,2.345679,0.283951,0.098765,0.000000,61.0,0.000000,0.000000,0.086420,0.0,0.0,1722.0,4052.0,729.0,0.0,11768.0,911.0,37590.0
3,83153,2020-05-09,2020-08-08,5,FEMENINO,Europe,1,58182,85,0.751882,0.848961,80.0,30.0,14.0,36.0,68.976471,914.0,623.0,0.705072,0.094118,4.577465,4.602941,0.376471,0.094118,0.000000,60.0,0.000000,0.000000,0.000000,0.0,0.0,5648.0,3990.0,0.0,0.0,6158.0,15424.0,20699.0
4,91963,2020-06-09,2020-09-08,7,MASCULINO,Europe,0,70215,77,0.828026,0.900704,84.0,52.0,28.0,4.0,60.831169,118.0,108.0,0.819474,0.727273,4.741935,4.160714,0.584416,0.233766,0.038961,52.0,0.012987,0.012987,0.000000,0.0,0.0,5032.0,2062.0,0.0,0.0,7964.0,3051.0,27468.0


In [18]:

# BASIC EDA for students (excluding alumno, tutor_principal)
var = ['fecha_inicio_primer_contrato', 'fecha_fin_primer_contrato',
       'edad', 'genero', 'zone', 'renueva', 'id_sesion',
       'efectividad', 'tmr', 'unidades_superadas',
       'unidades_superadas_sesiones', 'unidades_superadas_test_avance',
       'unidades_superadas_test_entrada', 'num_problemas',
       'num_problemas_corregidos', 'num_problemas_correctos_corregidos',
       'efectividad_correccion', 'mundo_virtual', 'pregunta_inicio',
       'pregunta_fin', 'racha_20', 'racha_30', 'racha_50',
       'completado_plan_estudios', 'num_partidas', 'num_juegos',
       'num_competiciones', 'tiempo_dedicado_conteo_y_numeros_cardinales',
       'tiempo_dedicado_expresiones_y_ecuaciones', 'tiempo_dedicado_geometria',
       'tiempo_dedicado_medicion_y_datos', 'tiempo_dedicado_numeros_enteros',
       'tiempo_dedicado_numeros_y_operaciones__decimales',
       'tiempo_dedicado_numeros_y_operaciones__en_base_diez',
       'tiempo_dedicado_numeros_y_operaciones__fracciones',
       'tiempo_dedicado_operaciones_y_pensamiento_algebraico']
report_student_data = sv.analyze(prueba[var])
report_student_data.show_html('report_student_data.html')


:FEATURES DONE:                    |█████████████████████| [100%]   00:11  -> (00:00 left)
:PAIRWISE DONE:                    |█████████████████████| [100%]   00:00  -> (00:00 left)


Creating Associations graph... DONE!
Report report_student_data.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.


In [21]:
# BASIC EDA for all sessions 
var = ['fecha', 'dia_contrato',
       'estado_sesion', 'tipo_sesion', 'efectividad', 'tmr',
       'unidades_superadas', 'unidades_superadas_sesiones',
       'unidades_superadas_test_avance', 'unidades_superadas_test_entrada',
       'estado_correccion', 'num_problemas_corregidos',
       'num_problemas_correctos_corregidos', 'efectividad_correccion',
       'mundo_virtual', 'calificacion', 'num_problemas', 'pregunta_inicio',
       'pregunta_fin', 'tipo_app', 'racha_20', 'racha_30', 'racha_50',
       'contenido_avanzado', 'completado_plan_estudios', 'num_partidas',
       'num_juegos', 'num_competiciones',
       'tiempo_dedicado_conteo_y_numeros_cardinales',
       'tiempo_dedicado_expresiones_y_ecuaciones', 'tiempo_dedicado_geometria',
       'tiempo_dedicado_medicion_y_datos', 'tiempo_dedicado_numeros_enteros',
       'tiempo_dedicado_numeros_y_operaciones__decimales',
       'tiempo_dedicado_numeros_y_operaciones__en_base_diez',
       'tiempo_dedicado_numeros_y_operaciones__fracciones',
       'tiempo_dedicado_operaciones_y_pensamiento_algebraico', 'dia_semana',
       'hora_inicio', 'hora_fin', 'zone']

report_all_data = sv.analyze(data2[var])
report_all_data.show_html('report_all_data.html')


                                   |                         | [  0%]   00:00  -> (? left)
Summarizing dataframe:             |                         | [  0%]   00:00  -> (? left)
Summarizing dataframe:             |▌                    | [  2%]   00:01  -> (00:57 left)
:fecha:                            |▌                    | [  2%]   00:01  -> (00:57 left)
:dia_contrato:                     |█                    | [  5%]   00:01  -> (00:55 left)
:dia_contrato:                     |█▌                   | [  7%]   00:01  -> (00:40 left)
:estado_sesion:                    |█▌                   | [  7%]   00:01  -> (00:40 left)
:estado_sesion:                    |██                   | [ 10%]   00:02  -> (00:29 left)
:tipo_sesion:                      |██                   | [ 10%]   00:02  -> (00:29 left)
:tipo_sesion:                      |██▌                  | [ 12%]   00:02  -> (00:22 left)
:efectividad:                      |██▌                  | [ 12%]   00:02  -> (00:22 left

:Processing Pairwise Features:     |█                    | [  5%]   00:00  -> (00:04 left)
:Processing Pairwise Features:     |█▌                   | [  7%]   00:14  -> (02:45 left)
:Processing Pairwise Features:     |██                   | [ 10%]   00:28  -> (04:27 left)
:Processing Pairwise Features:     |██▌                  | [ 12%]   00:28  -> (03:04 left)
:Processing Pairwise Features:     |███                  | [ 15%]   00:28  -> (02:08 left)
:Processing Pairwise Features:     |███▌                 | [ 17%]   00:29  -> (01:29 left)
:Processing Pairwise Features:     |████                 | [ 20%]   00:29  -> (01:03 left)
:Processing Pairwise Features:     |████▌                | [ 22%]   00:41  -> (02:41 left)
:Processing Pairwise Features:     |█████                | [ 24%]   00:42  -> (01:52 left)
:Processing Pairwise Features:     |█████▋               | [ 27%]   00:56  -> (03:27 left)
:Processing Pairwise Features:     |██████▏              | [ 29%]   00:56  -> (02:21 left)

Creating Associations graph... DONE!
Report report_all_data.html was generated! NOTEBOOK/COLAB USERS: no browser will pop up, the report is saved in your notebook/colab files.
